# Simple Lorenz attractor in Julia

In [139]:
using BenchmarkTools
#using Plots

In [2]:
using DifferentialEquations

┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1242


In [4]:
# Parameters
sigma = 10.0
rho = 28.0
beta = 8/3.;

In [5]:
# Fixed points
p1 = [-sqrt(beta*(rho-1)), -sqrt(beta*(rho-1)), rho-1];
p2 = [+sqrt(beta*(rho-1)), +sqrt(beta*(rho-1)), rho-1];

In [6]:
function lorenz!(du, u, p, t)
    du[1] = sigma*(u[2]-u[1])
    du[2] = u[1]*(rho-u[3]) - u[2]
    du[3] = u[1]*u[2] - beta*u[3]
end

lorenz! (generic function with 1 method)

In [141]:
# Cube size
xy_half_size = 25
z_size = 45
voxel_size = z_size/500.0;

In [143]:
# Convert x coordinate to one integer
to_index(x, left, right, n) = trunc(Int, (x - left)/(right - left)*n)

# test
to_index(12, -10, 10, 100)

110

In [144]:
# Same for the 3 coordinates
function to_ijk(xyz)
    x, y, z = xyz
    i = to_index(x, -xy_half_size, +xy_half_size, trunc(Int, 2*xy_half_size/voxel_size))
    j = to_index(y, -xy_half_size, +xy_half_size, trunc(Int, 2*xy_half_size/voxel_size))
    k = to_index(z, 0, +z_size, trunc(Int, z_size/voxel_size))
   return  i, j, k
end

# test
to_ijk(rand(3).*10)

(289, 333, 93)

In [145]:
# To the attractor
function burn()
    u0 = (rand(Float64, 3) .- [0.5, 0.5, 0]).*[2*xy_half_size, 2*xy_half_size, z_size]
    tspan = (0.0, 20.0)
    prob = ODEProblem(lorenz!, u0, tspan)
    tol = 1e-3
    sol = solve(prob, Tsit5(), reltol=tol, abstol=tol);
    return sol[end]
end

burn (generic function with 1 method)

In [146]:
# Default dict/Counter
function increment!(counter, ijk)
    counter[ijk] = get(counter, ijk, 0) + 1
    end;

# test
# increment!(cube, (1, 1, 2));

In [147]:
# Run the solver and fill the cube
function populate!(cube)
    u0 = burn()

    tspan = (0.0, 200.0)
    prob = ODEProblem(lorenz!, u0, tspan)

    tol = 1e-7
    sol = solve(prob, Tsit5(), reltol=tol, abstol=tol);

    #println("sol size: ", size(sol.u))

    for xyz in sol.u
        ijk = to_ijk(xyz)
        increment!(cube, ijk)
    end
    
end

populate! (generic function with 1 method)

In [121]:
# Init the counter
cube = Dict{Tuple{Int, Int, Int}, Int}()

Dict{Tuple{Int64,Int64,Int64},Int64} with 0 entries

In [132]:
for k in 1:100
    populate!(cube)
    print("\rcube size:  ", length(cube), "")
end
println("\n min, max= ", extrema(values(cube)))

cube size:  419057
 min, max= (1, 90)


In [131]:
varinfo(r"cube")

| name |       size | summary                                                  |
|:---- | ----------:|:-------------------------------------------------------- |
| cube | 33.000 MiB | Dict{Tuple{Int64,Int64,Int64},Int64} with 396223 entries |


In [138]:
# Save to text file
outfile = "data/lorenz_v02.xyz"
open(outfile, "w") do file
    for (ijk, count) in cube
        if count >= 2
            println(file, "$(ijk[1]) $(ijk[2]) $(ijk[3])")
        end
    end
end

Next job is for PotreeConvertor:
- [Install PotreeConvertor](https://github.com/potree/PotreeConverter)

        cmake -DCMAKE_BUILD_TYPE=Release -DLASZIP_INCLUDE_DIRS=~/Projets/strange_attractor/lastools/LASzip/dll -DLASZIP_LIBRARY=~/Projets/strange_attractor/lastools/LASzip/build/src/liblaszip.so ..
        # copy ./PotreeConverter/resources/page_template to your binary working directory.
        
        
- Path to use     PotreeConverter:

        export PATH="../PotreeConvertor/master/build/PotreeConverter/:$PATH"

- Command to convert:

        PotreeConverter -p --page-template ./page_template/ -i lorenz_v02.xyz
    
        PotreeConverter -f xyzrgb -color-range 1 -a RGB  -i thomas_pointcloud.xyz
        
and for [Potree](http://potree.org/):

--> [a white Lorenz Attractor](https://xdze2.github.io/chaotic-clouds/white_lorenz.html)


![screenshot](./images/thewhitebutterfly.png)

## Draft
using events

In [ ]:
n = [0, 0, 1]

function condition(u, t, integrator)
    a =  u .- p2
    return sum(a.*n)
end

function project(u)
    a =  u .- p2
    dot = sum(a.*n)
    return a .- dot.*n
end

affect!(integrator) = terminate!(integrator)
cb = ContinuousCallback(condition, affect!);

In [ ]:
u0 = burn()

tspan = (0.0, 60.0)
prob = ODEProblem(lorenz!, u0, tspan)

tol = 1e-7
sol_stop = solve(prob, Tsit5(), callback=cb, reltol=tol, abstol=tol);

In [ ]:
plot(sol[1, :], sol[3, :], xlabel="x", ylabel="z")

In [ ]:
plot(sol[2, :], sol[3, :], xlabel="y", ylabel="z")
scatter!([p1[2], ], [p1[3], ], label="p1")
scatter!([p2[2], ], [p2[3], ], label="p2")
plot!(sol_stop[2, :], sol_stop[3, :], color=:red)

In [ ]:
function to_plan()
    
    u0 = burn()
    tspan = (0.0, 160.0)
    prob = ODEProblem(lorenz!, u0, tspan)

    tol = 1e-7
    sol_stop = solve(prob, Tsit5(), callback=cb,
        reltol=tol, abstol=tol);
    return project(sol_stop[end])[1:2]
end

In [ ]:
poincarre = hcat([to_plan() for i in 1:5000]...);

In [ ]:
scatter(poincarre[1, :], poincarre[2, :], markersize=2, markerstrokewidth = 0,
            markeralpha = 0.6, xlabel="y", ylabel="x")
xaxis!(lims=(0, 5))

In [ ]:
u0 = burn()

tspan = (0.0, 1000.0)
prob = ODEProblem(lorenz!, u0, tspan)

tol = 1e-7
sol = solve(prob, Tsit5(), reltol=tol, abstol=tol);

In [ ]:
outfile = "data/lorenz_v01.xyz"
open(outfile, "w") do file
  for s in sol.u
    println(file, "$(s[1]) $(s[2]) $(s[3])")
  end
end

In [ ]:
CSV.write("hello.txt", rand(3, 3))

In [ ]:
?yaxis!#(lims=(1, 6))

In [ ]:
@gif for theta in range(0, stop = 360, length = 150)
    plot3d(sol[1, :], sol[2, :], sol[3, :],
           camera=(theta, 10))
end

In [ ]:
plot(sol[1, :], sol[3, :])

In [ ]:
u0 = [1.0; 0.0; 0.0]
tspan = (0.0, 40.0)
prob = ODEProblem(lorenz!, u0, tspan)
sol1 = solve(prob);
sol2 = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8);

In [ ]:
sol_OZ4 = solve(prob, OwrenZen4());
plot(sol_OZ4, linewidth=1)

In [ ]:
sol_RK4 = solve(prob, RK4(), reltol=1e-8, abstol=1e-8);
plot(sol_RK4, linewidth=1)

In [ ]:
plot(sol_RK4[2, :], sol_RK4[3, :])

In [ ]:
plot(sol_RK4.u[], linewidth=1)

In [ ]:
sol_RK4 = solve(prob, OwrenZen3(), reltol=1e-8, abstol=1e-8);
plot(sol_RK4, linewidth=1)

In [ ]:
sol_RK4 = solve(prob, Feagin14(), reltol=1e-8, abstol=1e-8);
plot(sol_RK4, linewidth=1)